In [ ]:
!pip install tensorflow pandas

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten,Dropout

In [4]:
train_df = pd.read_csv('/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/train/_annotations.csv')
test_df = pd.read_csv('/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/test/_annotations.csv')
valid_df = pd.read_csv('/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/valid/_annotations.csv')


In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [6]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/train',
    x_col='filename',
    y_col='class',
    target_size=(180, 180),
    batch_size=64,
    class_mode='categorical'
)

Found 28518 validated image filenames belonging to 6 classes.


In [7]:
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory='/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/valid',
    x_col='filename',
    y_col='class',
    target_size=(180, 180),
    batch_size=64,
    class_mode='categorical'
)

Found 3581 validated image filenames belonging to 6 classes.


In [8]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/kaggle/input/brackish-tf/Brackish 960x540.tensorflow/test',
    x_col='filename',
    y_col='class',
    target_size=(180, 180),
    batch_size=64,
    class_mode='categorical'
)


Found 3466 validated image filenames belonging to 6 classes.


In [9]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(180, 180, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(460, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(58, activation='relu')(x)
x = Dense(188, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# 
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // valid_generator.batch_size,
    epochs=10
)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/445 ━━━━━━━━━━━━━━━━━━━━ 5:25:18 44s/step - accuracy: 0.1719 - loss: 4.3071

I0000 00:00:1718022335.028138     146 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1718022335.049850     146 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


161/445 ━━━━━━━━━━━━━━━━━━━━ 2:12 467ms/step - accuracy: 0.4981 - loss: 2.0361

...

445/445 ━━━━━━━━━━━━━━━━━━━━ 205s 362ms/step - accuracy: 0.5798 - loss: 1.4233 - val_accuracy: 0.7563 - val_loss: 0.6511
Epoch 2/10
  1/445 ━━━━━━━━━━━━━━━━━━━━ 1:12 164ms/step - accuracy: 0.7031 - loss: 0.7378

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


445/445 ━━━━━━━━━━━━━━━━━━━━ 38s 85ms/step - accuracy: 0.7031 - loss: 0.7378 - val_accuracy: 0.6885 - val_loss: 0.6925
Epoch 3/10


W0000 00:00:1718022533.585229     147 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


445/445 ━━━━━━━━━━━━━━━━━━━━ 109s 242ms/step - accuracy: 0.7374 - loss: 0.6470 - val_accuracy: 0.7548 - val_loss: 0.5887
Epoch 4/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.7656 - loss: 0.5249 - val_accuracy: 0.8197 - val_loss: 0.4992
Epoch 5/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 110s 244ms/step - accuracy: 0.7559 - loss: 0.5884 - val_accuracy: 0.7645 - val_loss: 0.5563
Epoch 6/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7031 - loss: 0.6695 - val_accuracy: 0.6721 - val_loss: 0.6162
Epoch 7/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 112s 247ms/step - accuracy: 0.7599 - loss: 0.5649 - val_accuracy: 0.7741 - val_loss: 0.5070
Epoch 8/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - accuracy: 0.9375 - loss: 0.3455 - val_accuracy: 0.8033 - val_loss: 0.3966
Epoch 9/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 110s 244ms/step - accuracy: 0.7700 - loss: 0.5293 - val_accuracy: 0.7844 - val_loss: 0.4969
Epoch 10/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.7969 - loss: 0.5756 - va

In [12]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)


54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 292ms/step - accuracy: 0.7957 - loss: 0.4603
Test loss: 0.474257230758667
Test accuracy: 0.7893518805503845


In [13]:
import matplotlib.pyplot as plt

def plot_history(history, title):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{title} Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{title} Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()


In [14]:
# Plot history for VGG16
plot_history(model_vgg16, 'VGG16')

NameError: name 'model_vgg16' is not defined

In [15]:
# Save the VGG16 model
model.save('Vgg16_Brackish(78.93%).h5')